In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
## RFM => Regency (Último -> data - data - 1) / Frequency (quantidade -> n) / Monetary (soma/média -> R   ~x)

def rfm_variables(df):
    ## FREQUENCY
    f_sales = len(df) ## Quantidade de vendas 
    f_profit = len(df[df['Profit'] > 0]) ## Quantidade de vendas com lucros
    ## MONETARY
    m_sales = round(df['Sales'].sum(), 2) ## Total de vendas
    m_profit = round(df['Profit'].sum(), 2) ## Total de vendas com lucro
    m_quantity = df['Quantity'].sum() ## Total de quantidades
    ## PERIODICITY
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates() ## Periodicidade
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    ## REGENCY
    r_days = round(df_sort['diff_int'].mean(), 2)
    return f_sales, f_profit, m_sales, m_profit, m_quantity, r_days

In [3]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variables = data[variable].unique()
    for v in variables:
        var = data[data[variable] == v]
        f_sales, f_profit, m_sales, m_profit, m_quantity, r_days = rfm_variables(var)
        rfm = rfm.append(
            {
                'reference': v,
                'm_sales': m_sales,
                'm_profit': m_profit, 
                'm_quantity': m_quantity,
                'r_days': r_days,
                'f_sales': f_sales, 
                'f_profit': f_profit
            },
            ignore_index = True
        )
    return rfm

In [4]:
data = pd.read_feather('database/base.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Region,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,...,6,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,Miami,Florida,South,...,1,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1


In [5]:
variables = ['m_sales', 'm_profit', 'm_quantity', 'r_days', 'f_sales', 'f_profit']

In [6]:
state_rfm = fit_data(data, 'State')
state_rfm = state_rfm.fillna(0)
state_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    state_rfm[
        variables
    ]
).labels_

state_rfm

/tmp/ipykernel_1659/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1659/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1659/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1659/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1659/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1659/3999741325.py:7: FutureWarning

,reference,m_sales,m_profit,m_quantity,r_days,f_sales,f_profit,cluster
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0
1,California,457687.63,76381.39,7667,2.21,2001,1884,1
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0
4,Washington,138641.27,33402.65,1883,6.54,506,483,2
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0
7,Utah,11220.06,2546.53,219,53.16,53,53,0
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2


In [7]:
cluster = pd.DataFrame()

for index, row in enumerate(KMeans(n_clusters=3, random_state=0).fit(state_rfm[variables]).cluster_centers_):
    cluster = cluster.append(
        {
            'cluster': index,
            'clf_sales': row[0],
            'clf_profit': row[1],
            'clm_sales': row[2],
            'clm_profit': row[3],
            'clm_quantity': row[4],
            'clr_days': row[5]
        },
        ignore_index= True 
    ) 

cluster

/tmp/ipykernel_1659/1360041999.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_1659/1360041999.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_1659/1360041999.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(


,cluster,clf_sales,clf_profit,clm_sales,clm_profit,clm_quantity,clr_days
0,0.0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
1,1.0,384281.950000,75209.970000,5945.500000,2.780000,1564.500,1479.000000
2,2.0,102518.188750,274.487500,1822.750000,7.898750,487.625,312.375000


In [8]:
state_rfm = state_rfm.merge(cluster, on='cluster', how='left')
state_rfm

,reference,m_sales,m_profit,m_quantity,r_days,f_sales,f_profit,cluster,clf_sales,clf_profit,clm_sales,clm_profit,clm_quantity,clr_days
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
1,California,457687.63,76381.39,7667,2.21,2001,1884,1,384281.950000,75209.970000,5945.500000,2.780000,1564.500,1479.000000
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2,102518.188750,274.487500,1822.750000,7.898750,487.625,312.375000
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
4,Washington,138641.27,33402.65,1883,6.54,506,483,2,102518.188750,274.487500,1822.750000,7.898750,487.625,312.375000
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2,102518.188750,274.487500,1822.750000,7.898750,487.625,312.375000
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
7,Utah,11220.06,2546.53,219,53.16,53,53,0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0,18166.447179,3430.287179,292.307692,77.675641,76.000,66.692308
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2,102518.188750,274.487500,1822.750000,7.898750,487.625,312.375000


In [ ]:
state_rfm.to_feather('database/clusterizacao_estado.feather')